<a href="https://colab.research.google.com/github/pranav-696/ML-Repo./blob/main/DFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import os
import shutil
import dlib
from PIL import Image

In [ ]:
img_file = "Frame_img"
crop_image_file = "crop_img"

if(os.path.isdir(img_file) == True):
    shutil.rmtree(img_file)
os.mkdir(img_file)

if(os.path.isdir(crop_image_file) == True):
    shutil.rmtree(crop_image_file)
os.mkdir(crop_image_file)

In [ ]:
video_path = '/content/id56_0005.mp4'

if(os.path.isdir(img_file) == True):
    shutil.rmtree(img_file)
os.mkdir(img_file)

capture = cv2.VideoCapture(video_path)
f = 0
intrval = 12
save_frame = 0
while(capture.isOpened()):
    ret, frame = capture.read()
    if ret == False:
        break
    if f % intrval == 0:
        cv2.imwrite('Frame_img/imgf'+str(f)+'.jpg',frame)
        save_frame += 1
    f += 1

In [ ]:
def detect_and_crop_face_dlib(image_path, output_path):
    # Load the image using dlib
    img = dlib.load_rgb_image(image_path)

    # Initialize dlib's face detector (HOG-based)
    detector = dlib.get_frontal_face_detector()

    # Detect faces in the image
    faces = detector(img, 1)

    # Check if any faces are detected
    if len(faces) == 0:
        print("No faces detected")
        return None

    # Loop over the detected faces
    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()

        # Crop the face from the image
        cropped_face = Image.fromarray(img[y:y+h, x:x+w])

        # Save the cropped face
        cropped_face.save(output_path)

        # Return the cropped face
        return cropped_face

# Example usage
k= 10
j = 7
for filename in os.listdir(img_file):
  if(filename.endswith('.jpg') == True):
    cropped_face = detect_and_crop_face_dlib('Frame_img/'+filename, crop_image_file+'/img_crop'+str(k)+str(j)+'.jpg' )
    if cropped_face is not None:
        print("Face cropped and saved successfully.")
    k+=1

Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
Face cropped and saved successfully.
F

In [ ]:
# for filename in os.listdir(crop_image_file):
#     if filename.endswith('.jpg'):
#         new_filename = "h_056" + filename
#         # Construct the full file paths
#         old_file = os.path.join(crop_image_file, filename)
#         new_file = os.path.join(crop_image_file, new_filename)

#         # Rename the file
#         os.rename(old_file, new_file)

In [ ]:
# from google.colab import files

# # Step 1: Zip the folder
# shutil.make_archive(crop_image_file, 'zip', '/content/crop_img' )

# # Step 2: Download the zipped folder
# files.download('crop_img.zip')

MAIN CODE IS FROM HERE ON

In [ ]:
f_imgs = []

for filename in os.listdir(crop_image_file):
  if filename.endswith('.jpg'):
    filepath = os.path.join(crop_image_file, filename)
    img = Image.open(filepath)
    f_imgs.append({
        'filename': filename,
        'filepath': filepath,
        'width' : img.width,
        'height' : img.height
    })

In [ ]:
import numpy as np

def pixel_error(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # represent the image in grey scale
  laplacin_measure = cv2.Laplacian(gray, cv2.CV_64F).var()
  return int(laplacin_measure < 100)

In [ ]:
def shadow_inconsistncies(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                      cv2.THRESH_BINARY_INV, 11, 2)

    kernel = np.ones((5, 5), np.uint8)
    clean_mask = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)

    shadow_pixels = np.sum(clean_mask == 255)

    shadow_pixel_threshold = 0.2 * (image.shape[0] * image.shape[1])

    if shadow_pixels > shadow_pixel_threshold:
        return 0  # Shadow inconsistency detected
    else:
        return 1

In [ ]:
import dlib

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def face_landmark_misalignment(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = detector(gray)
  misalignment_detected =[]

  for face in faces:
    landmarks = predictor(gray, face)
    landmarks_np = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(68)])

    left_eye = landmarks_np[36:42]
    right_eye = landmarks_np[42:48]
    left_eye_center = np.mean(left_eye, axis=0)
    right_eye_center = np.mean(right_eye, axis=0)
    eye_asymmetry = abs(left_eye_center[1] - right_eye_center[1])

    interocular_distance = np.linalg.norm(left_eye_center - right_eye_center)
    eye_asymmetry_threshold = 0.05 * interocular_distance

    if eye_asymmetry > eye_asymmetry_threshold:
        misalignment_detected.append(1) #'Eye Asymmetry'

    nose_tip = landmarks_np[30]
    face_center_x = (landmarks_np[0, 0] + landmarks_np[16, 0])
    nose_deviation = abs(nose_tip[0] - face_center_x)

    face_width = landmarks_np[16, 0] - landmarks_np[0, 0]
    nose_deviation_threshold = 0.1 * face_width

    if nose_deviation > nose_deviation_threshold:
        misalignment_detected.append(2) #'Nose Deviation'

    left_mouth_corner = landmarks_np[48]
    right_mouth_corner = landmarks_np[54]
    mouth_skewness = abs(left_mouth_corner[1] - right_mouth_corner[1])

    mouth_width = np.linalg.norm(left_mouth_corner - right_mouth_corner)
    mouth_skewness_threshold = 0.1 * mouth_width

    if mouth_skewness > mouth_skewness_threshold:
        misalignment_detected.append(3) #'Mouth Skewness'

    chin_point = landmarks_np[8]  # Chin point
    chin_deviation = abs(chin_point[0] - face_center_x)

    chin_deviation_threshold = 0.08 * face_width

    if chin_deviation > chin_deviation_threshold:
        misalignment_detected.append(4) #'Chin Displacement'

  return misalignment_detected

In [ ]:
!pip install pyngrok

In [ ]:
pixel_anomaly = {}
shadow_anomaly = {}
landmark_anomaly = {}

for img_info in f_imgs:
  image = cv2.imread(img_info['filepath'])
  if image is not None:
    pixel_anomaly= pixel_error(image)
    shadow_anomaly = shadow_inconsistncies(image)
    landmark_anomaly = face_landmark_misalignment(image)

    Eye_Asymmetry = 0
    Nose_Deviation = 0
    Mouth_Skewness = 0
    Chin_Displacement = 0

    for i in landmark_anomaly:
      if(i==1):
        Eye_Asymmetry = 1
      elif(i==2):
        Nose_Deviation = 1
      elif(i==3):
        Mouth_Skewness = 1
      elif(i==4):
        Chin_Displacement = 1


    img_info.update({
        'pixelation': pixel_anomaly,
        'shadow_inconsistency': shadow_anomaly,
        'Eye_Asymmetry': Eye_Asymmetry,
        'Nose_Deviation': Nose_Deviation,
        'Mouth_Skewness': Mouth_Skewness,
        'Chin_Displacement': Chin_Displacement
    })
  else:
    print("Image not found")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:


real_images_dir = '/content/drive/MyDrive/Train_img/real'
fake_images_dir = '/content/drive/MyDrive/Train_img/fake'

image_list = []

for filename in os.listdir(real_images_dir):
  if filename.endswith('.jpg'):
    filepath = os.path.join(real_images_dir, filename)

    img = cv2.imread(filepath)
    img_dimention = Image.open(filepath)

    pixel_anomaly= pixel_error(img)
    shadow_anomaly = shadow_inconsistncies(img)
    landmark_anomaly = face_landmark_misalignment(img)

    Eye_Asymmetry = 0
    Nose_Deviation = 0
    Mouth_Skewness = 0
    Chin_Displacement = 0
    img = Image.open(filepath)

    if(len(landmark_anomaly)!=0):
      for i in landmark_anomaly:
        if(i==1):
          Eye_Asymmetry = 1
        elif(i==2):
          Nose_Deviation = 1
        elif(i==3):
          Mouth_Skewness = 1
        elif(i==4):
          Chin_Displacement = 1

    image_list.append({
        'filename': filename,
        'image_path': filepath,
        'width' : img_dimention.width,
        'height' : img_dimention.height,
        'real_fake_label' : 0,
        'pixelation': pixel_anomaly,
        'shadow_inconsistency': shadow_anomaly,
        'eye_asymmetry': Eye_Asymmetry,
        'nose_deviation': Nose_Deviation,
        'mouth_skewness': Mouth_Skewness,
        'chin_displacement': Chin_Displacement
    })


for filename in os.listdir(fake_images_dir):
  if filename.endswith('.jpg'):
    filepath = os.path.join(fake_images_dir, filename)

    img = cv2.imread(filepath)
    img_dimention = Image.open(filepath)

    pixel_anomaly= pixel_error(img)
    shadow_anomaly = shadow_inconsistncies(img)
    landmark_anomaly = face_landmark_misalignment(img)

    Eye_Asymmetry = 0
    Nose_Deviation = 0
    Mouth_Skewness = 0
    Chin_Displacement = 0

    if(len(landmark_anomaly)!=0):
      for i in landmark_anomaly:
        if(i==1):
          Eye_Asymmetry = 1
        elif(i==2):
          Nose_Deviation = 1
        elif(i==3):
          Mouth_Skewness = 1
        elif(i==4):
          Chin_Displacement = 1

    image_list.append({
        'filename': filename,
        'image_path': filepath,
        'width' : img_dimention.width,
        'height' : img_dimention.height,
        'real_fake_label' : 1,
        'pixelation': pixel_anomaly,
        'shadow_inconsistency': shadow_anomaly,
        'eye_asymmetry': Eye_Asymmetry,
        'nose_deviation': Nose_Deviation,
        'mouth_skewness': Mouth_Skewness,
        'chin_displacement': Chin_Displacement
    })


In [ ]:
import pandas as pd
df = pd.DataFrame(image_list)

In [ ]:
df.head()

,filename,image_path,width,height,real_fake_label,pixelation,shadow_inconsistency,eye_asymmetry,nose_deviation,mouth_skewness,chin_displacement
0,r_356img_crop695.jpg,/content/drive/MyDrive/Train_img/real/r_356img...,156,156,0,1,0,1,1,1,1
1,r_356img_crop738.jpg,/content/drive/MyDrive/Train_img/real/r_356img...,268,269,0,1,1,0,0,0,0
2,r_356img_crop739.jpg,/content/drive/MyDrive/Train_img/real/r_356img...,186,187,0,1,0,1,0,1,1
3,r_356img_crop737.jpg,/content/drive/MyDrive/Train_img/real/r_356img...,269,268,0,1,0,1,0,1,1
4,r_356img_crop658.jpg,/content/drive/MyDrive/Train_img/real/r_356img...,187,187,0,1,0,1,1,1,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1374 entries, 0 to 1373
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   filename              1374 non-null   object
 1   image_path            1374 non-null   object
 2   width                 1374 non-null   int64 
 3   height                1374 non-null   int64 
 4   real_fake_label       1374 non-null   int64 
 5   pixelation            1374 non-null   int64 
 6   shadow_inconsistency  1374 non-null   int64 
 7   eye_asymmetry         1374 non-null   int64 
 8   nose_deviation        1374 non-null   int64 
 9   mouth_skewness        1374 non-null   int64 
 10  chin_displacement     1374 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 118.2+ KB


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence

class DataFrameGenerator(Sequence):
    def __init__(self, df, batch_size, img_size, shuffle=True):
        self.df = df
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.df) // self.batch_size

    def __getitem__(self, index):
        batch_df = self.df.iloc[index * self.batch_size:(index + 1) * self.batch_size]
        images = []
        real_fake_labels = []
        pixelation_labels = []
        shadow_labels = []
        eye_asymmetry_labels = []
        mouth_skewness_labels = []
        nose_deviation_labels = []
        chin_displacement_labels = []

        for _, row in batch_df.iterrows():
            img = load_img(row['image_path'], target_size=self.img_size)
            img = img_to_array(img) / 255.0  # Normalize the image

            images.append(img)
            real_fake_labels.append(row['real_fake_label'])
            pixelation_labels.append(row['pixelation'])
            shadow_labels.append(row['shadow_inconsistency'])
            eye_asymmetry_labels.append(row['eye_asymmetry'])
            mouth_skewness_labels.append(row['mouth_skewness'])
            nose_deviation_labels.append(row['nose_deviation'])
            chin_displacement_labels.append(row['chin_displacement'])

        return np.array(images), {
            'real_fake_output': np.array(real_fake_labels),
            'pixelation_output': np.array(pixelation_labels),
            'shadow_output': np.array(shadow_labels),
            'eye_asymmetry_output': np.array(eye_asymmetry_labels),
            'mouth_skewness_output': np.array(mouth_skewness_labels),
            'nose_deviation_output': np.array(nose_deviation_labels),
            'chin_displacement_output': np.array(chin_displacement_labels)
        }

    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input

# Define the CNN model with multiple outputs including specific facial abnormalities
def create_multi_error_detection_model():
    inputs = Input(shape=(128, 128, 3))

    # Shared layers
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Output 1: Real/Fake classification
    real_fake_output = Dense(1, activation='sigmoid', name='real_fake_output')(x)

    # Output 2: Pixelation classification
    pixelation_output = Dense(1, activation='sigmoid', name='pixelation_output')(x)

    # Output 3: Shadow Inconsistency classification
    shadow_output = Dense(1, activation='sigmoid', name='shadow_output')(x)

    # Output 4: Eye Asymmetry classification
    eye_asymmetry_output = Dense(1, activation='sigmoid', name='eye_asymmetry_output')(x)

    # Output 5: Mouth Skewness classification
    mouth_skewness_output = Dense(1, activation='sigmoid', name='mouth_skewness_output')(x)

    # Output 6: Nose Deviation classification
    nose_deviation_output = Dense(1, activation='sigmoid', name='nose_deviation_output')(x)

    # Output 7: Chin Displacement classification
    chin_displacement_output = Dense(1, activation='sigmoid', name='chin_displacement_output')(x)

    # Define the model
    model = Model(inputs=inputs, outputs=[real_fake_output, pixelation_output, shadow_output, eye_asymmetry_output, mouth_skewness_output, nose_deviation_output, chin_displacement_output])

    # Compile the model with separate loss functions
    model.compile(
        optimizer='adam',
        loss={
            'real_fake_output': 'binary_crossentropy',
            'pixelation_output': 'binary_crossentropy',
            'shadow_output': 'binary_crossentropy',
            'eye_asymmetry_output': 'binary_crossentropy',
            'mouth_skewness_output': 'binary_crossentropy',
            'nose_deviation_output': 'binary_crossentropy',
            'chin_displacement_output': 'binary_crossentropy'
        },
        metrics={
            'real_fake_output': 'accuracy',
            'pixelation_output': 'accuracy',
            'shadow_output': 'accuracy',
            'eye_asymmetry_output': 'accuracy',
            'mouth_skewness_output': 'accuracy',
            'nose_deviation_output': 'accuracy',
            'chin_displacement_output': 'accuracy'
        }
    )

    return model

# Create the model
model = create_multi_error_detection_model()


In [ ]:
# Parameters
batch_size = 32
img_size = (128, 128)
epochs = 10

# Create the DataFrame generator
train_generator = DataFrameGenerator(df, batch_size, img_size)

model.summary()

# Train the model
model.fit(train_generator, epochs=epochs)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 128, 128, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 126, 126, 32)   │            896 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 63, 63, 32)     │              0 │ conv2d[0][0]           │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 61, 61, 64)     │         18,496 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 30, 30, 64)     │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 28, 28, 128)    │         73,856 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 14, 14, 128)    │              0 │ conv2d_2[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 25088)          │              0 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │      3,211,392 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 128)            │              0 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ real_fake_output (Dense)  │ (None, 1)              │            129 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pixelation_output (Dense) │ (None, 1)              │            129 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ shadow_output (Dense)     │ (None, 1)              │            129 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ eye_asymmetry_output      │ (None, 1)              │            129 │ dropout[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ mouth_skewness_output     │ (None, 1)              │            129 │ dropout[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ nose_deviation_output     │ (None, 1)              │            129 │ dropout[0][0]          │
│ (Dense)                   │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ chin_displacement_out

 Total params: 3,305,543 (12.61 MB)

 Trainable params: 3,305,543 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


42/42 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - chin_displacement_output_accuracy: 0.8721 - eye_asymmetry_output_accuracy: 0.5525 - loss: 3.2684 - mouth_skewness_output_accuracy: 0.5332 - nose_deviation_output_accuracy: 0.9355 - pixelation_output_accuracy: 0.9704 - real_fake_output_accuracy: 0.5872 - shadow_output_accuracy: 0.8807
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - chin_displacement_output_accuracy: 0.9457 - eye_asymmetry_output_accuracy: 0.5874 - loss: 2.8005 - mouth_skewness_output_accuracy: 0.5218 - nose_deviation_output_accuracy: 0.9361 - pixelation_output_accuracy: 0.9723 - real_fake_output_accuracy: 0.5957 - shadow_output_accuracy: 0.9933
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - chin_displacement_output_accuracy: 0.9468 - eye_asymmetry_output_accuracy: 0.6607 - loss: 2.6124 - mouth_skewness_output_accuracy: 0.5080 - nose_deviation_output_accuracy: 0.9335 - pixelation_output_accuracy: 0.9738 - real_fake_output_accuracy: 0.6669 - shadow_output_accuracy: 0.9937


In [ ]:
def predict_image(model, image_path):
    img = load_img(image_path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image

    predictions = model.predict(img_array)

    real_fake_prob = predictions[0][0]
    pixelation_prob = predictions[1][0]
    shadow_prob = predictions[2][0]
    nose_deviation_prob = predictions[3][0]
    chin_displacement_prob = predictions[4][0]

    # Determine Real/Fake label
    real_fake_label = 'Real' if real_fake_prob > 0.5 else 'Fake'
    real_fake_confidence = real_fake_prob if real_fake_prob > 0.5 else 1 - real_fake_prob

    # Error probabilities
    error_probabilities = {
        'Pixelation': pixelation_prob * 100,
        'Shadow Inconsistency': shadow_prob * 100,
        'Eye Asymmetry': 100 - (pixelation_prob + shadow_prob) * 100,
        'Mouth Skewness': 100 - (pixelation_prob + shadow_prob) * 100,
        'Nose Deviation': nose_deviation_prob * 100,
        'Chin Displacement': chin_displacement_prob * 100
    }

    return real_fake_label, real_fake_confidence * 100, error_probabilities


In [ ]:
# Predict on a sample image
real_fake_ratio = []
for img_meta in f_imgs:
   image_path = img_meta['filepath']
   real_fake_label, real_fake_confidence, error_probabilities = predict_image(model, image_path)
   real_fake_confidence = real_fake_confidence.item()
   real_fake_ratio.append(real_fake_label)
   print(f"Label: {real_fake_label}, Confidence: {real_fake_confidence:.2f}%")
   print("Error Probabilities:", error_probabilities)
print(real_fake_ratio)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
Label: Real, Confidence: 62.24%
Error Probabilities: {'Pixelation': array([99.91405], dtype=float32), 'Shadow Inconsistency': array([0.19404912], dtype=float32), 'Eye Asymmetry': array([-0.10810089], dtype=float32), 'Mouth Skewness': array([-0.10810089], dtype=float32), 'Nose Deviation': array([47.561417], dtype=float32), 'Chin Displacement': array([12.929704], dtype=float32)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Label: Real, Confidence: 82.39%
Error Probabilities: {'Pixelation': array([99.80105], dtype=float32), 'Shadow Inconsistency': array([0.20560767], dtype=float32), 'Eye Asymmetry': array([-0.00666046], dtype=float32), 'Mouth Skewness': array([-0.00666046], dtype=float32), 'Nose Deviation': array([56.3854], dtype=float32), 'Chin Displacement': array([36.683697], dtype=float32)}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Label: Real, Confidence: 80.56%
Error Probabilities: {'Pixelation': array([99.944336], dtype=float32), 'Shadow Inconsistency': 

In [ ]:
real_c = 0
fake_c = 0
for i in real_fake_ratio:
  if(i=='Real'):
    real_c+=1
  else:
    fake_c+=1
print("real persentage : ",real_c/len(real_fake_ratio) * 100)
print("fake persentage : ",fake_c/len(real_fake_ratio) * 100)

real persentage :  87.5
fake persentage :  12.5


In [ ]:
import pickle

# Load the .pkl model
with open('/content/drive/MyDrive/model.pkl', 'rb') as file:
    model = pickle.load(file)

In [ ]:
from tensorflow.keras.models import save_model

# Save the model in .h5 format
model.save('/content/drive/MyDrive/model.h5')  # Save it as .h5 format

# Alternatively, save in .keras format
model.save('/content/drive/MyDrive/model.keras')  # Save it as .keras format

In [ ]:
!pip install flask-ngrok
!pip install flask
!pip install dlib
!pip install tensorflow
!pip install pillow
!apt-get install cmake
!pip install face_recognition
!pip install opencv-python-headless

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=bcce2cd3b4d731822d841da81225fff999af76ca19c99270588f50699a905872
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [ ]:
!apt-get install ngrok  # Install Ngrok
!ngrok authtoken "2lVh1lhns8BObYcyJgVN7F2QA7t_6MdTZ4u6dgyA3GDkee2L3"
!ngrok http 5000

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package ngrok
/bin/bash: line 1: ngrok: command not found
/bin/bash: line 1: ngrok: command not found


In [ ]:
import os
import shutil
import cv2
import dlib
import numpy as np
from PIL import Image
from flask import Flask, request, redirect, jsonify
from flask_ngrok import run_with_ngrok
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from pyngrok import ngrok

app = Flask(__name__)
run_with_ngrok(app)  # Start ngrok when the app is run

# Load the pre-trained model
model = load_model('/content/drive/MyDrive/model.keras')

# Define directories for saving images
img_file = "Frame_img"
crop_image_file = "crop_img"

if os.path.isdir(img_file):
    shutil.rmtree(img_file)
os.mkdir(img_file)

if os.path.isdir(crop_image_file):
    shutil.rmtree(crop_image_file)
os.mkdir(crop_image_file)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

def detect_and_crop_face_dlib(image_path, output_path):
    img = dlib.load_rgb_image(image_path)
    faces = detector(img, 1)

    if len(faces) == 0:
        return None

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        cropped_face = Image.fromarray(img[y:y+h, x:x+w])
        cropped_face.save(output_path)
        return cropped_face

def process_video(video_path):
    capture = cv2.VideoCapture(video_path)
    f = 0
    intrval = 12
    save_frame = 0
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            break
        if f % intrval == 0:
            cv2.imwrite(f'{img_file}/imgf{f}.jpg', frame)
            save_frame += 1
        f += 1

    k = 0
    for filename in os.listdir(img_file):
        if filename.endswith('.jpg'):
            detect_and_crop_face_dlib(f'{img_file}/{filename}', f'{crop_image_file}/frame-{k}.jpg')
            k += 1

def predict_image(model, image_path):
    img = load_img(image_path, target_size=(128, 128))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = img_array / 255.0  # Normalize the image

    predictions = model.predict(img_array)

    # Convert predictions to scalar values
    real_fake_prob = float(predictions[0][0])
    pixelation_prob = float(predictions[1][0])
    shadow_prob = float(predictions[2][0])
    eye_asymmetry_prob = float(predictions[3][0])
    mouth_skewness_prob = float(predictions[4][0])
    nose_deviation_prob = float(predictions[5][0])
    chin_displacement_prob = float(predictions[6][0])

    real_fake_label = 'Real' if real_fake_prob > 0.5 else 'Fake'
    real_fake_confidence = real_fake_prob if real_fake_prob > 0.5 else 1 - real_fake_prob

    error_probabilities = {
        'Pixelation': pixelation_prob * 100,
        'Shadow Inconsistency': shadow_prob * 100,
        'Eye Asymmetry': eye_asymmetry_prob * 100,
        'Mouth Skewness': mouth_skewness_prob * 100,
        'Nose Deviation': nose_deviation_prob * 100,
        'Chin Displacement': chin_displacement_prob * 100
    }

    return real_fake_label, real_fake_confidence * 100, error_probabilities

@app.route('/')
def home():
    return '''
        <html lang="en">
<head>
    <link rel="stylesheet" type="text/css" href="/static/styles.css">
    <script src="/static/scripts.js"></script>
</head>
<body>
    <div class="home">
        <h1>Upload a Video</h1>
        <form action="/upload" method="post" enctype="multipart/form-data">
            <input type="file" name="file" accept="video/*">
            <input type="submit" value="Upload">
        </form>
    </div>
</body>
</html>
    '''

@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return redirect(request.url)
    file = request.files['file']
    if file.filename == '':
        return redirect(request.url)
    if file:
        video_path = os.path.join("/content", file.filename)
        file.save(video_path)
        process_video(video_path)

        results = []
        total_fake_confidence = 0
        total_real_confidence = 0
        fake_count = 0
        real_count = 0

        for filename in os.listdir(crop_image_file):
            if filename.endswith('.jpg'):
                image_path = os.path.join(crop_image_file, filename)
                real_fake_label, real_fake_confidence, error_probabilities = predict_image(model, image_path)
                results.append({
                    'filename': filename,
                    'real_fake_label': real_fake_label,
                    'real_fake_confidence': real_fake_confidence,
                    'error_probabilities': error_probabilities
                })

                if real_fake_label == 'Fake':
                    total_fake_confidence += real_fake_confidence
                    fake_count += 1
                else:
                    total_real_confidence += real_fake_confidence
                    real_count += 1

        # Calculate the average confidence for fake and real labels
        avg_fake_confidence = total_fake_confidence / fake_count if fake_count > 0 else 0
        avg_real_confidence = total_real_confidence / real_count if real_count > 0 else 0

        # Generate styled HTML output
        output = '''
        <style>
            body {
                font-family: Arial, sans-serif;
                background-color: #c3d4e389;
                margin: 0;
                padding: 0;


            }
            .container {
                width: 80%;
                margin: auto;
                background-color: #fff;
                padding: 20px;
                box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
                border-radius: 8px;
                margin-top: 30px;
                display:flex;
                flex-direction:column;
                align-items:center;
            }
            h2 {
                color: #333;
                text-align: center;
                border-bottom: 2px solid #ddd;
                padding-bottom: 10px;
                margin-bottom: 20px;
                color: #343a40;
                background-image: linear-gradient(to top ,rgb(31, 83, 253),rgb(31, 179, 253));
                -webkit-background-clip: text;
                -webkit-text-fill-color: transparent;
            }
            .cart {
                list-style-type: none;
                padding: 0;
                display: flex;
                flex-wrap: wrap;
                align-items:center;
                justify-content:space-around;
                margin: 0;
                gap: 10px; /* Add spacing between items */
            }
            .cart li {
                flex: 1 0 calc(33.333% - 20px); /* Adjust width for three columns with space for gap */
                box-sizing: border-box;
                padding: 10px;
                margin-bottom: 20px;
                background-color: #f9f9f9;
                border: 1px solid #ddd;
                border-radius: 5px;
            }
            .cart li:nth-child(odd) {
                background-color: #e9e9e9; /* Alternate background color */
            }
            .cart li:nth-child(even) {
                background-color: #f4f4f4; /* Alternate background color */
            }
            ul, li ul {
                list-style: none;
                padding: 0;
                margin: 0;
            }
            li ul li {
                background-color: #fff;
                border: none;
                padding: 5px;
            }
            h3 {
                color: #555;
                text-align: center;
                margin-top: 20px;
            }
        </style>
        <div class="container">
            <h2>Results:</h2>
        '''
        output += f"<h3>Average Fake Confidence: {avg_fake_confidence:.2f}%</h3>"
        output += "<ul class='cart'>"
        for result in results:
            output += "<ul>"
            output += f"<li>{result['filename']} - <strong>{result['real_fake_label']}</strong> (Confidence: {result['real_fake_confidence']:.2f}%)"
            if result['error_probabilities']:
                output += "<ul>"
                for error, prob in result['error_probabilities'].items():
                    output += f"<li>{error}: {prob:.2f}%</li>"
                output += "</ul>"
            output += "</li>"
            output += "</ul>"

        output += "</ul>"
        #output += f"<h3>Average Fake Confidence: {avg_fake_confidence:.2f}%</h3>"
        #output += f"<h3>Average Real Confidence: {avg_real_confidence:.2f}%</h3>"
        output += "</div>"

        return output

# Open a tunnel on port 5000
authtoken = "2lYEWnzYTbSdtqk1UH2WnQx2O7y_TNywAcFM1rP4Gr8yQxjP"  # Use your ngrok auth token
ngrok.set_auth_token(authtoken)
public_url = ngrok.connect(5000)
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:5000\"")

# Run the Flask app
if __name__ == '__main__':
    app.run()

 * ngrok tunnel "NgrokTunnel: "https://5adf-34-91-221-99.ngrok-free.app" -> "http://localhost:5000"" -> "http://127.0.0.1:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cda2-34-91-221-99.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:41:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:41:48] "GET /static/scripts.js HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:41:48] "GET /static/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:41:50] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step


<ipython-input-50-13f88bf1a5c3>:76: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  real_fake_prob = float(predictions[0][0])
<ipython-input-50-13f88bf1a5c3>:77: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pixelation_prob = float(predictions[1][0])
<ipython-input-50-13f88bf1a5c3>:78: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  shadow_prob = float(predictions[2][0])
<ipython-input-50-13f88bf1a5c3>:79: DeprecationWarning: Conversion of an array with ndim > 0 to a s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


INFO:werkzeug:127.0.0.1 - - [03/Sep/2024 17:43:23] "POST /upload HTTP/1.1" 200 -


In [ ]:
from pyngrok import ngrok

# Terminate open tunnels if any
ngrok.kill()

# Create a new tunnel
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

Public URL: NgrokTunnel: "https://fc79-34-91-221-99.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
# Download the Ngrok package
!wget -q -O ngrok.zip https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

# Unzip the downloaded file
!unzip ngrok.zip

In [ ]:
# Replace YOUR_AUTH_TOKEN with your actual Ngrok token
!./ngrok authtoken "2lVh1lhns8BObYcyJgVN7F2QA7t_6MdTZ4u6dgyA3GDkee2L3"